In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import pickle
import os
import datetime

# Load data
data = pd.read_csv('/content/drive/MyDrive/ELM/merged_CSV/merged_132K.csv').dropna()

X = data.drop('Class', axis=1)
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Models
rf_final = RandomForestClassifier(n_estimators=500, random_state=0)
gb_final = GradientBoostingClassifier(n_estimators=500, random_state=0)
xgb_final = xgb.XGBClassifier(n_estimators=1000, random_state=0)

ensemble_model = VotingClassifier(estimators=[('RF', rf_final), ('GB', gb_final), ('XGB', xgb_final)], voting='soft')

models = [ensemble_model]
model_names = ["Ensemble"]

# Feature Combinations
Combination_t1_band_list = ['s2_red', 's2_blue', 's2_green', 'Class']
Combination_t2_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'Class']
Combination_t3_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'Class']
Combination_t4_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'VV', 'VH', 'HV', 'HH', 'Class']
Combination_t5_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'VV', 'VH', 'HV', 'HH', 'HV_savg', 'VV_savg', 'VH_savg','HH_savg', 'Class']
Combination_t5_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'VV', 'VH', 'HV', 'HH', 'HV_savg', 'VV_savg', 'VH_savg','HH_savg', 'viirs', 'Class']
Combination_t6_band_list = ['ndviMax', 'viirs', 's2_nir', 'ndviSD', 's2_swir2', 's2_swir1', 'mndwiMedian', 'VV_savg', 'VH', 'HH_savg', 'HV_savg',
    'VH_savg', 'nduiMin', 'SISAI', 's2_aerosols', 's2_red', 'HV', 'VV_dent', 'NSAI1', 'nduiMedian', 'VV', 'VV_idm',
    's2_green', 'ndbiMin', 's2_blue', 'NSAI2', 'ndbiMedian', 'HH', 'mndwiMax', 'VH_dent', 'VH_idm', 'HH_dent', 'HH_idm',
    'HV_dent', 'HV_idm', 'Class']
Combination_t7_band_list  = ['ndviMax', 'viirs', 'ndviSD', 'mndwiMedian', 's2_swir2', 's2_nir', 's2_swir1', 'VV_savg', 'HV_savg', 'SISAI', 'VH_savg',
 'VV', 'nduiMin', 'VH', 'HH_savg', 'NSAI1', 's2_aerosols', 's2_red', 'nduiMedian', 'HV', 's2_green', 'ndbiMedian',
 'VV_dent', 'ndbiMin', 'HH', 's2_blue', 'NSAI2', 'mndwiMax', 'VV_idm', 'VV_sent', 'VH_dent', 'VV_imcorr1', 'VH_idm',
 'VH_sent', 'VH_imcorr1', 'HH_dent', 'VH_corr', 'VV_corr', 'HH_idm', 'HV_dent', 'HV_idm', 'HH_corr', 'HH_sent', 'HV_sent',
 'HH_imcorr1', 'HV_imcorr1', 'HV_corr', 'Class']
combinations = [Combination_t1_band_list , Combination_t2_band_list , Combination_t3_band_list , Combination_t4_band_list , Combination_t5_band_list , Combination_t6_band_list , Combination_t7_band_list ]
combinations_names = ['Combination1', 'Combination2', 'Combination3', 'Combination4', 'Combination5', 'Combination6', 'Combination7']

for i in range(len(combinations)):

  X_combo = data[combinations[i]]

  for j in range(len(models)):

    # Get the current date and time
    now = datetime.datetime.now()

    # Format the date and time
    date_time_suffix = now.strftime("%Y-%m-%d_%H-%M-%S")

    # Update the filename
    filename = combinations_names[i] + "_" + model_names[j] + "_" + date_time_suffix + ".pkl"

    X_train, X_test, y_train, y_test = train_test_split(X_combo, y, test_size=0.2, random_state=42)

    model = models[j]
    model.fit(X_train, y_train)

    # Save model
    save_dir = "/content/drive/MyDrive/ELM/TrainedModel"
    filepath = os.path.join(save_dir, filename)
    pickle.dump(model, open(filepath, 'wb'))

    # Evaluate
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print(f"{combinations_names[i]} + {model_names[j]} Accuracy: {accuracy}")

Combination1 + Ensemble Accuracy: 1.0
Combination2 + Ensemble Accuracy: 1.0
Combination3 + Ensemble Accuracy: 1.0
Combination4 + Ensemble Accuracy: 1.0
Combination5 + Ensemble Accuracy: 1.0
Combination6 + Ensemble Accuracy: 1.0
Combination7 + Ensemble Accuracy: 1.0


In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

data = pd.read_csv('/content/drive/MyDrive/ELM/merged_CSV/merged_132K.csv').dropna()

X = data.drop( 'Class', axis=1)
y = data[ 'Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Models
rf_final = RandomForestClassifier(n_estimators=500,  random_state=0)
# ab_final = AdaBoostClassifier(n_estimators=500,  random_state=0)
gb_final = GradientBoostingClassifier(n_estimators=500,  random_state=0)
xgb_final = xgb.XGBClassifier(n_estimators=1000,  random_state=0)

ensemble_model = VotingClassifier(estimators=[('RF', rf_final), ('GB', gb_final), ('XGB', xgb_final)], voting='soft') #('AB', ab_final),

models = [
  # rf_final, gb_final, xgb_final,
  ensemble_model] #ab_final,
model_names = [
  # "RF",  "GB", "XGB",
  "Ensemble"] #"AB",

# Feature Combinations
Combination_t1_band_list = ['s2_red', 's2_blue', 's2_green', 'Class']
Combination_t2_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'Class']
Combination_t3_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'Class']
Combination_t4_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'VV', 'VH', 'HV', 'HH', 'Class']
Combination_t5_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'VV', 'VH', 'HV', 'HH', 'HV_savg', 'VV_savg', 'VH_savg','HH_savg', 'Class']
Combination_t5_band_list = ['s2_red', 's2_blue', 's2_green', 's2_nir', 's2_swir1', 's2_swir2', 'ndviMax', 'mndwiMedian', 'ndviSD', 'SISAI', 'nduiMin', 's2_aerosols', 'nduiMedian', 'ndbiMedian', 'NSAI2', 'ndbiMin', 'NSAI1', 'mndwiMax', 'VV', 'VH', 'HV', 'HH', 'HV_savg', 'VV_savg', 'VH_savg','HH_savg', 'viirs', 'Class']
Combination_t6_band_list = ['ndviMax', 'viirs', 's2_nir', 'ndviSD', 's2_swir2', 's2_swir1', 'mndwiMedian', 'VV_savg', 'VH', 'HH_savg', 'HV_savg',
    'VH_savg', 'nduiMin', 'SISAI', 's2_aerosols', 's2_red', 'HV', 'VV_dent', 'NSAI1', 'nduiMedian', 'VV', 'VV_idm',
    's2_green', 'ndbiMin', 's2_blue', 'NSAI2', 'ndbiMedian', 'HH', 'mndwiMax', 'VH_dent', 'VH_idm', 'HH_dent', 'HH_idm',
    'HV_dent', 'HV_idm', 'Class']
Combination_t7_band_list  = ['ndviMax', 'viirs', 'ndviSD', 'mndwiMedian', 's2_swir2', 's2_nir', 's2_swir1', 'VV_savg', 'HV_savg', 'SISAI', 'VH_savg',
 'VV', 'nduiMin', 'VH', 'HH_savg', 'NSAI1', 's2_aerosols', 's2_red', 'nduiMedian', 'HV', 's2_green', 'ndbiMedian',
 'VV_dent', 'ndbiMin', 'HH', 's2_blue', 'NSAI2', 'mndwiMax', 'VV_idm', 'VV_sent', 'VH_dent', 'VV_imcorr1', 'VH_idm',
 'VH_sent', 'VH_imcorr1', 'HH_dent', 'VH_corr', 'VV_corr', 'HH_idm', 'HV_dent', 'HV_idm', 'HH_corr', 'HH_sent', 'HV_sent',
 'HH_imcorr1', 'HV_imcorr1', 'HV_corr', 'Class']
combinations = [Combination_t1_band_list , Combination_t2_band_list , Combination_t3_band_list , Combination_t4_band_list , Combination_t5_band_list , Combination_t6_band_list , Combination_t7_band_list ]
combinations_names = ['Combination1', 'Combination2', 'Combination3', 'Combination4', 'Combination5', 'Combination6', 'Combination7']

# Evaluate models and combinations
for i in range(len(combinations)):

  X_combo = data[combinations[i]] # filter features

  for j in range(len(models)):

    X_train, X_test, y_train, y_test = train_test_split(X_combo, y, test_size=0.2, random_state=42)
    models[j].fit(X_train, y_train)
    pred = models[j].predict(X_test)

    # if j < len(models)-1:
    accuracy = accuracy_score(y_test, pred)
    # print(f"{model_names[j]} Accuracy: {accuracy}")
    print(f"{combinations_names[i]} + {model_names[j]} Accuracy: {accuracy}")

    # print(f"{combinations_names[i]} + {model_names[j]}")
    # print(classification_report(y_test, pred))

Combination1 + Ensemble Accuracy: 1.0
Combination2 + Ensemble Accuracy: 1.0
Combination3 + Ensemble Accuracy: 1.0
Combination4 + Ensemble Accuracy: 1.0
Combination5 + Ensemble Accuracy: 1.0
Combination6 + Ensemble Accuracy: 1.0
Combination7 + Ensemble Accuracy: 1.0
